Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

In [7]:
data = pd.read_csv('/datasets/insurance.csv')
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

### Вопрос: признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

**Ответ:** Не изменится

**Обоснование:**

Введем следующие обозначения:

$X_p = X P$ - матрица признаков после "шифрования", вектор целевого признака $y$ шифровать не будем.

Используя следующие свойства операций с матрицами:

$$(A B)^T = B^T A^T$$

$$(A B)^{-1} = B^{-1} A^{-1}$$

$$(A^T)^{-1} = (A^{-1})^T$$

а также формулу обучения, получаем следующую формулу предсказания для зашифрованных данных:

$$a_p = X_p w_p = X_p (X_p^T X_p)^{-1} X_p^T y =$$

$$= (X P)\,((X P)^T X P)^{-1} (X P)^T y =$$

$$= (X P)\,(P^T (X^T X) P)^{-1} P^T X^T y =$$

$$= (X P)\,P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y =$$

$$= X (P P^{-1}) (X^T X)^{-1} (P P^{-1})^T X^T y =$$

$$= X (X^T X)^{-1} X^T y =$$

$$= X w =$$

$$= a$$

Получили $a_p = a$

**Вывод: при шифровании признаков $X$ обратимой матрицей $P$ предсказания $a$ не меняются.**

Как связаны параметры $w$ и $w_p$:

выше мы получили соотношение $X_p w_p = X w$. Средовательно:

$$X P\,w_p = X w$$

$$w = P\,w_p$$

или

$$w_p = P^{-1} w$$

## 3. Алгоритм преобразования

Описал в п.2

Исходная матрица признаков $X$ преобразуется ("шифруется") следующим образом: матрица умножается на произвольную обратимую матрицу $P$: 

$$X_p = X P$$

Для обратного преобразования (получения матрицы $X$) необходимо умножить матрицу $X_p$ на $P^{-1}$: 

$$X_p P^{-1} = X P P^{-1} = X E = X$$

## 4. Проверка алгоритма

In [8]:
# Введем свой класс 
# Обращаю внимание, что для удобства на вход должны подавать матрицы и вектора

class LinearRegression:
    def fit(self, X, y):
        X1 = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
        w = np.linalg.inv(X1.T @ X1) @ X1.T @ y
        self.w = w[1:]
        self.w0 = w[0]
        
    def predict(self, X):
        return X @ self.w + self.w0

In [9]:
x_train = data.drop('Страховые выплаты', axis=1)
y_train = data['Страховые выплаты']

In [11]:
model1 = LinearRegression()
model1.fit(x_train.values, y_train.values)
pred1 = model1.predict(x_train.values)

In [16]:
# Сгенерируем произвольную обратимую матрицу нужных размеров
# Используем равномерное распределение
state = np.random.RandomState(12345)
P = state.uniform(0, 1, (x_train.shape[1], x_train.shape[1]))
print(P)

try:
    np.linalg.inv(P)
    print('Матрица обратима')
except:
    print('Матрица необратима!')

[[0.92961609 0.31637555 0.18391881 0.20456028]
 [0.56772503 0.5955447  0.96451452 0.6531771 ]
 [0.74890664 0.65356987 0.74771481 0.96130674]
 [0.0083883  0.10644438 0.29870371 0.65641118]]
Матрица обратима


In [84]:
# И строим новую модель, используя для обучения XP
model2 = LinearRegression()
XP = x_train.values @ P
model2.fit(XP, y_train.values)
pred2 = model2.predict(XP)

In [85]:
print(pred1[0:20])
print(pred2[0:20])

[ 0.51172715  0.68431581  0.09373365 -0.22258878  0.06508374  0.57103879
  0.42842668 -0.09843783  0.33316862  0.18999524 -0.05739468  0.42162253
 -0.16436565 -0.19771553  0.46909652  0.2798098  -0.03734603  0.50094881
  0.56098611  0.02753857]
[ 0.51172025  0.6843275   0.09373945 -0.22257042  0.06506908  0.57103325
  0.4284212  -0.09843524  0.33316219  0.18998964 -0.05740312  0.4216077
 -0.16434446 -0.19770115  0.46911008  0.27979394 -0.0373482   0.50094165
  0.56097561  0.02752649]


Даже на глазок видно, что предсказанные значения у исходной модели и модели с шифрованием идентичны. Различия в 5-ых, 6-ых знаках после запятой связываю с округлением.

In [86]:
print('Метрика R2 для обычной модели: ', r2_score(y_train, pred1))
print('Метрика R2 для модели c шифрованием: ', r2_score(y_train, pred2))

Метрика R2 для обычной модели:  0.4249455028666801
Метрика R2 для модели c шифрованием:  0.4249455021696207


Метрики R2 практически идентичны.